# Setup

In [ ]:
from __future__ import absolute_import, division, print_function

import os
import sys
import re
from collections import Counter
from itertools import chain

import numpy as np
import pandas as pd

from IPython.display import display
from ipywidgets import FloatProgress

%matplotlib inline

pd.set_option('display.max_columns', 100)

# Use whatever path is convenient for you
CLINVAR_RAW_DATA_DIR = os.path.join(os.path.expanduser('~'), 'data/clinvar')

# Change to the path where your project is cloned
FIRM_DATA_DIR = os.path.join(os.path.expanduser('~'), 'github_projects/firm/firm/data')

FINAL_DATASET_FEATURES_FILE_PATH = os.path.join(FIRM_DATA_DIR, 'clinvar_final_dataset_features.csv.gz')
CLASSIFIER_DUMP_FILE_PATH = os.path.join(FIRM_DATA_DIR, 'classifier.pkl')

In [ ]:
def summarize(df, n = 5):
    display(df.head(n))
    print('%d records' % len(df))
    
def update_progress_bar(progress_bar, i, sensitivity):
    if i % sensitivity == 0 or i == progress_bar.max:
        progress_bar.value = i

In [ ]:
import geneffect
import firm

REFERENCE_GENOME = 'GRCh37'

geneffect_setup = geneffect.Setup(REFERENCE_GENOME)
firm.setup_uniprot_tracks(geneffect_setup)

# Load and prepare ClinVar dataset

** NOTE: If you just want to load the pre-extracted features, you can skip this entire section. **

In [ ]:
# Download ClinVar's full dataset from their FTP website at:
# ftp://ftp.ncbi.nlm.nih.gov/pub/clinvar/tab_delimited/variant_summary.txt.gz
clinvar_data = pd.read_csv(os.path.join(CLINVAR_RAW_DATA_DIR, 'variant_summary.txt.gz'), delimiter = '\t', \
        na_values = ['-', 'na'])
summarize(clinvar_data)

In [ ]:
# Filter only SNPs with the correct reference genome
clinvar_snps = clinvar_data.loc[(clinvar_data['Type'] == 'single nucleotide variant') & (clinvar_data['Assembly'] == \
        REFERENCE_GENOME), ['Chromosome', 'Start', 'ReferenceAllele', 'AlternateAllele', \
        'ClinicalSignificance']].copy().reset_index(drop = True)
summarize(clinvar_snps)

In [ ]:
# Use the variant processing framework to processes the SNPs

total_failures = 0
progress_bar = FloatProgress(min = 0, max = len(clinvar_snps) - 1)
display(progress_bar)

def process_snp_from_record(snp_record):
    
    update_progress_bar(progress_bar, snp_record.name, 1000)
    
    try:
        return geneffect_setup.variant_interpreter.process_snp(str(snp_record['Chromosome']), int(snp_record['Start']), \
                snp_record['ReferenceAllele'], snp_record['AlternateAllele'])
    except:
        global total_failures
        total_failures += 1
        return np.nan
            
clinvar_snps['processed_snp'] = clinvar_snps.apply(process_snp_from_record, axis = 1)
print('%d of %d failed.' % (total_failures, len(clinvar_snps)))
summarize(clinvar_snps)

In [ ]:
# Filter only missense variants

def is_missense(snp):
    return pd.notnull(snp) and len(snp.gene_effects) == 1 and snp.gene_effects[0].is_missense()

clinvar_missense_snps = clinvar_snps.loc[clinvar_snps['processed_snp'].apply(is_missense)].copy()
summarize(clinvar_missense_snps)

In [ ]:
# Determine pathogenicity of the variants and drop duplicates

SPLIT_REGEX = re.compile('[/;]')
PATHOGENIC_KEYWORDS = set(['pathogenic', 'likely pathogenic'])
BENIGN_KEYWORDS = set(['benign', 'likely benign'])

def determine_pathogenicity(keywords):
    if keywords.intersection(PATHOGENIC_KEYWORDS):
        return 1
    elif keywords.intersection(BENIGN_KEYWORDS):
        return 0
    else:
        return np.nan

def parse_clinical_significance(significance):
    if pd.isnull(significance) or significance.isdigit():
        return set()
    else:
        return set([keyword.lower() for keyword in SPLIT_REGEX.split(significance)])

pathogenicity_keywords = clinvar_missense_snps['ClinicalSignificance'].apply(parse_clinical_significance)
keyword_counter = Counter(chain.from_iterable(pathogenicity_keywords))
print('There are %d unique Clinical Significance keywords: %s' % (len(keyword_counter), str(keyword_counter)))

clinvar_missense_snps['is_pathogenic'] = pathogenicity_keywords.apply(determine_pathogenicity)
clinvar_missense_snps['is_pathogenic'].value_counts(dropna = False, normalize = True).plot.pie(\
        figsize = (4, 4), autopct = '%.f%%')

final_clinvar_dataset = clinvar_missense_snps.dropna(subset = ['is_pathogenic']).drop_duplicates(['Chromosome', 'Start', \
        'ReferenceAllele', 'AlternateAllele', 'is_pathogenic'])
summarize(final_clinvar_dataset)

# Extract features

In [ ]:
# Extracting features. This is a long process (~2 hours)...

from firm.variant_feature_extraction import FeatureExtractionSetup, get_snp_effect_feature_extractor

feature_extraction_setup = FeatureExtractionSetup(geneffect_setup)
snp_effect_feature_extractor = get_snp_effect_feature_extractor(feature_extraction_setup)

records = final_clinvar_dataset
records_gene_effect = records['processed_snp'].apply(lambda snp: snp.gene_effects[0])

%time features = snp_effect_feature_extractor.create_features_as_dataframe(records_gene_effect, show_progress_bar = True)
features.insert(0, 'label', list(records['is_pathogenic']))
features.insert(0, 'seq', list(records_gene_effect.apply(lambda gene_effect: \
        str(gene_effect.affected_gene.uniprot_record.seq))))
features.insert(0, 'snp_effect', list(records_gene_effect))

summarize(features, 15)

In [ ]:
# Override the CSV file of the final dataset features with the ones you have just extracted.
# WARNING: This will actually override the project's CSV file. Consider saving to another path if you don't want to do that.
features.to_csv(FINAL_DATASET_FEATURES_FILE_PATH, index = False)

In [ ]:
# Or, just load the pre-extracted features.
# WARNING: If you have already run the above cells, this will override everything!
features = pd.read_csv(FINAL_DATASET_FEATURES_FILE_PATH)
summarize(features, 15)

In [ ]:
X_records = features.loc[:, 'protein_length':]
feature_names = np.array(X_records.columns)
X = X_records.as_matrix()
y = features['label'].as_matrix()

print(X.shape, X.dtype)
print(y.shape, y.dtype)

# Validate the existing model

** NOTE: this will give training error (with overfitting), not test error! **

In [ ]:
from firm.ml.classification import predict_prob
from firm.ml.metric_helper import get_formatted_scores

firm_classifier = firm.load_classifier(geneffect_setup)

X_selected = firm_classifier.feature_selection.transform(X)
y_pred_prob = predict_prob(firm_classifier.model, X_selected)
y_pred = (y_pred_prob >= firm_classifier.default_prob_threshold)
print(get_formatted_scores(y, y_pred, y_pred_prob))

# Training the ML model

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import RandomForestClassifier

from firm.ml.cross_validation import cross_validate

### Setup ###

SEED = 7126

np.random.seed(SEED)


### Dataset statistics ###

n = len(y)
n_positive = sum(y)
n_negative = n - n_positive
f_positive = float(n_positive) / n
f_negative = 1 - f_positive

print('Dataset size: %d' % n)
print('Label imbalance: %f' % f_positive)


### Choosing feature selection method and model ###

feature_selection = VarianceThreshold()
model = RandomForestClassifier(n_estimators = 100, min_samples_split = 50, class_weight = 'balanced', n_jobs = -1, \
        random_state = SEED)

print('Model: %s' % model)
print('-' * 50)


### Cross validation ###

cross_validate(X, y, model, n_folds = 3, feature_selection = feature_selection, feature_names = feature_names, \
        report_removed_features = False, seed = SEED)

In [ ]:
# Dump the trained classifier.
# WARNING: This will actually override the project's classifier. Consider saving to another path if you don't want to do that.

from firm.ml.classification import Classifier

classifier = Classifier(get_snp_effect_feature_extractor, feature_extractor_creator_args = [feature_extraction_setup], \
        model = model, feature_selection = feature_selection)
classifier.train_on_processed_data(X, y)

with open(CLASSIFIER_DUMP_FILE_PATH, 'wb') as f:
    classifier.dump(f)
    
print('Done.')